In [569]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF



from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

import glob
import os

In [371]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [372]:
RANDOM_SEED = 6

In [763]:
DATA_PATH = Path.cwd() / ""
train_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
train_df['checkout_datetime'] = pd.to_datetime(train_df['drop_time'])
train_df['checkin_datetime'] = pd.to_datetime(train_df['pickup_time'])
#train_df['duration'] = train_df['checkout_datetime']-train_df['checkin_datetime']
train_df['duration_in_minutes'] = (train_df['checkout_datetime']-train_df['checkin_datetime'])/pd.Timedelta(minutes=1)
train_df['lat']  = abs(train_df['pick_lat']-train_df['drop_lat'])
train_df['lon']  = abs(train_df['pick_lon']-train_df['drop_lon'])
#train_df['distance'] = (((train_df['lat'])**2 + (train_df['lon'])**2)**(1/2))*100
#train_df['manhatton_distance'] = (train_df['lat'] + train_df['lat'])

#train_df.loc[train_df['meter_waiting_fare'] > 0, 'is_meter_waiting_fare'] = 1
#train_df.loc[train_df['meter_waiting_fare'] <= 0, 'is_meter_waiting_fare'] = 0

#train_df['meter_waiting_fare'] = train_df['meter_waiting_fare'].astype(np.int64)


#train_df['is_waiting_fare'] = train_df['meter_waiting']*train_df['meter_waiting_fare']
#train_df['waiting_till_pickup_fare'] = train_df['meter_waiting_till_pickup']*train_df['meter_waiting_fare']

train_df['date_time'] = pd.to_datetime(train_df['drop_time'])
train_df['drop_Hour'] = train_df['date_time'].apply(lambda x: x.hour+1)
#train_df['weekday'] = train_df['date_time'].apply(lambda x: x.weekday())
train_df['remaining_fare'] = train_df['fare']-train_df['additional_fare']-train_df['meter_waiting_fare']
#train_df['r_fare'] = train_df['fare']-train_df['additional_fare']
train_df['mobile_time'] = train_df['duration']-train_df['meter_waiting']-train_df['meter_waiting_till_pickup']
#train_df['m_time'] = train_df['duration']-train_df['meter_waiting']
#train_df['unknown'] = train_df['duration']-train_df['duration_in_minutes']*60
#train_df['mobile_fare_time'] = train_df['remaining_fare']-train_df['mobile_time']
#train_df['unknown_new'] = (train_df['lat']**2)+(train_df['lon']**2)





#train_df
#train_df['pick_Hour'] = pd.to_datetime(train_df['pickup_time'])
#train_df['pick_Hour'] = train_df['pick_Hour'].apply(lambda x: x.hour+1)

#features_df_num = train_df.drop(["date_time","pickup_time","drop_time","duration","checkin_datetime","drop_lat","drop_lon","checkout_datetime", "label"], axis = 1)
features_df = train_df.drop(["date_time","duration","pickup_time","drop_time","checkin_datetime","drop_lat","drop_lon","pick_lat","pick_lon","checkout_datetime", "label"], axis = 1)

pure_train_df = train_df.drop(["label"], axis = 1)
categorical_features = pure_train_df.columns[pure_train_df.dtypes == "object"].values


label_df = train_df[["label"]]
label_df = label_df.replace({'label': {'correct': 1, 'incorrect': 0}})

features_df.head()

,additional_fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,duration_in_minutes,lat,lon,drop_Hour,remaining_fare,mobile_time
tripid,,,,,,,,,,,
189123628,10.5,56.0,0.0000,64.0,270.32,14.0,0.04078,0.0210,1,259.8200,714.0
189125358,10.5,47.0,0.0000,134.0,197.85,13.0,0.02784,0.0061,2,187.3500,610.0
189125719,10.5,80.0,0.0000,61.0,301.64,18.0,0.02830,0.0495,2,291.1400,946.0
189127273,10.5,271.0,15.6638,68.0,82.30,10.0,0.00178,0.0076,3,56.1362,259.0
189128020,NaN,NaN,NaN,NaN,358.39,17.0,0.02963,0.0675,4,NaN,NaN


In [764]:
features_list = features_df.columns.tolist()
features_list

['additional_fare',
 'meter_waiting',
 'meter_waiting_fare',
 'meter_waiting_till_pickup',
 'fare',
 'duration_in_minutes',
 'lat',
 'lon',
 'drop_Hour',
 'remaining_fare',
 'mobile_time']

In [765]:
## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('minmax_scaler', MinMaxScaler())
])

## chain non-numerical preprocessing into a pipeline object
non_numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore')),
#     ('label_encoder', LabelEncoder())
])

## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_preprocessing_steps, features_list)
    ],
    remainder = 'drop'
)



In [778]:
#estimator =  RandomForestClassifier( class_weight='balanced', n_estimators=500, criterion="entropy")
#estimator =  RandomForestClassifier( class_weight='balanced', n_estimators=400)
estimator = GradientBoostingClassifier(loss='exponential',n_estimators=10000)
#estimator = KNeighborsClassifier(n_neighbors=100, weights="uniform", algorithm="brute")
#estimator = AdaBoostClassifier()
#estimator =  RandomForestClassifier( class_weight='balanced', n_estimators=400, max_depth=200, max_features=10)

In [779]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('estimators', estimator)
])


In [780]:
features_df

,additional_fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,duration_in_minutes,lat,lon,drop_Hour,remaining_fare,mobile_time
tripid,,,,,,,,,,,
189123628,10.5,56.0,0.000000,64.0,270.32,14.0,0.04078,0.0210,1,259.820000,714.0
189125358,10.5,47.0,0.000000,134.0,197.85,13.0,0.02784,0.0061,2,187.350000,610.0
189125719,10.5,80.0,0.000000,61.0,301.64,18.0,0.02830,0.0495,2,291.140000,946.0
189127273,10.5,271.0,15.663800,68.0,82.30,10.0,0.00178,0.0076,3,56.136200,259.0
189128020,NaN,NaN,NaN,NaN,358.39,17.0,0.02963,0.0675,4,NaN,NaN
189129552,10.5,182.0,0.000000,112.0,1065.02,57.0,0.21537,0.0320,7,1054.520000,3113.0
189132829,10.5,487.0,0.000000,133.0,266.62,20.0,0.00698,0.0427,7,256.120000,626.0
189135103,10.5,295.0,17.198500,212.0,318.05,22.0,0.00126,0.0482,8,290.351500,826.0
189139296,10.5,80.0,4.664000,3.0,100.32,6.0,0.00716,0.0060,8,85.156000,277.0


In [782]:
X_train, X_eval, y_train, y_eval = train_test_split(features_df, label_df, test_size=0.3, shuffle=True, stratify=label_df, random_state=RANDOM_SEED)

## Train the model
pipeline.fit(X_train, y_train)

# Predict for the evaluation set

print("Training Accuracy: %.2f" % (pipeline.score(X_eval, y_eval)*100), "%")
preds = pipeline.predict_proba(X_eval)


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Training Accuracy: 95.13 %


In [752]:
pipeline.fit(features_df, label_df)

None


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [753]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [754]:
test_set['checkout_datetime'] = pd.to_datetime(test_set['drop_time'])
test_set['checkin_datetime'] = pd.to_datetime(test_set['pickup_time'])
#train_df['duration'] = train_df['checkout_datetime']-train_df['checkin_datetime']
test_set['duration_in_minutes'] = (test_set['checkout_datetime']-test_set['checkin_datetime'])/pd.Timedelta(minutes=1)
test_set['lat']  = abs(test_set['pick_lat']-test_set['drop_lat'])
test_set['lon']  = abs(test_set['pick_lon']-test_set['drop_lon'])
#test_set.loc[test_set['meter_waiting_fare'] > 0, 'is_meter_waiting_fare'] = 1
#test_set.loc[test_set['meter_waiting_fare'] <= 0, 'is_meter_waiting_fare'] = 0
test_set['date_time'] = pd.to_datetime(test_set['drop_time'])
test_set['drop_Hour'] = test_set['date_time'].apply(lambda x: x.hour+1)
test_set['remaining_fare'] = test_set['fare']-test_set['additional_fare']-test_set['meter_waiting_fare']
test_set['mobile_time'] = test_set['duration']-test_set['meter_waiting']-test_set['meter_waiting_till_pickup']
#train_df['r_fare'] = train_df['fare']-train_df['additional_fare']
#train_df['m_time'] = train_df['duration']-train_df['meter_waiting']
#train_df['mobile_fare_time'] = train_df['remaining_fare']-train_df['mobile_time']




#test_set['distance'] = ((test_set['lat'])**2 + (test_set['lat'])**2)**(1/2)

new_test_set = test_set.drop(["date_time","duration","pickup_time","drop_time","pick_lat","pick_lon","checkin_datetime","drop_lat","drop_lon","checkout_datetime"], axis = 1)

#new_test_set = test_set.drop(["date_time","pickup_time", "drop_time","duration","meter_waiting_fare","drop_lat","drop_lon","checkin_datetime", "checkout_datetime"], axis = 1)
new_test_set

,additional_fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,duration_in_minutes,lat,lon,drop_Hour,remaining_fare,mobile_time
tripid,,,,,,,,,,,
213284604,10.5,42,2.448600,148,289.27,15.0,0.05964,0.0090,1,276.321400,734
213286352,10.5,20,0.000000,91,1912.70,71.0,0.36077,0.0983,3,1902.200000,4138
213293973,10.5,255,2.658800,23,394.00,26.0,0.01606,0.0511,6,380.841200,1274
213294622,10.5,16,0.000000,198,154.32,8.0,0.02968,0.0009,6,143.820000,248
213298687,10.5,392,12.369200,69,147.47,14.0,0.00907,0.0216,8,124.600800,353
213299545,10.5,351,16.530800,9,1156.97,42.0,0.14097,0.0652,8,1129.939200,2135
213302332,10.5,454,23.929200,43,196.81,18.0,0.02811,0.0019,9,162.380800,611
213302671,10.5,320,18.496000,17,688.43,45.0,0.00599,0.1045,9,659.434000,2400
213305594,10.5,29,0.000000,130,288.77,19.0,0.03188,0.0489,9,278.270000,995


In [755]:
test_probs = pipeline.predict_proba(new_test_set)


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [756]:
test_probs

array([[0.476, 0.524],
       [0.492, 0.508],
       [0.462, 0.538],
       ...,
       [0.246, 0.754],
       [0.312, 0.688],
       [0.374, 0.626]])

In [757]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()


,prediction
tripid,
213284604,1
213286352,0
213293973,0
213294622,1
213298687,1


In [758]:
try:
    np.testing.assert_array_equal(test_set.index.values, submission_set.index.values)
    print("rows in the same order")
except:
    print("rows not in the same order or error")


rows in the same order


In [759]:
submission_set['prediction'] = test_probs[:,1]


In [760]:
submission_set

,prediction
tripid,
213284604,0.524
213286352,0.508
213293973,0.538
213294622,0.588
213298687,0.574
213299545,0.542
213302332,0.554
213302671,0.436
213305594,0.610


In [761]:
submission_set.loc[submission_set['prediction'] > 0.5, 'prediction'] = 1
submission_set.loc[submission_set['prediction'] <= 0.5, 'prediction'] = 0
submission_set['prediction'] = submission_set['prediction'].astype(np.int64)


#submission_set['presiction'].convert_dtypes(convert_integer: bool = True)
submission_set



,prediction
tripid,
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,0
213305594,1


In [762]:
submission_set.to_csv('160374E_submission_10_95_42.csv', index=True)
print("Completed!")


Completed!
